In [180]:
# Warning control
import warnings
import openai
warnings.filterwarnings('ignore')

# Import OpenAI key
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()


In [182]:
import json
from IPython.display import display, Markdown, HTML
from openai import OpenAI
import re

client = OpenAI(api_key=openai_api_key)
O1_MODEL = 'o1-mini'

In [184]:
structured_prompt = ("<instructions>You are a support account manager. Your job is to create a root cause "
          "  analysis following critical system down event and share it with the customers.</instructions>\n"
          "<policy>**The root cause analysis contains**\n\n"
            "1. **Executive Summary**\n"
            "   - Include high-level and concise issue description, business impact and resolution \n"
            "2. **Technical Findings and Root cause**\n"
            "   - Symptoms \n"
            "   - Analysis \n"
            "   - Resolution/Workaround \n"
            "   - Issue Timeline \n"
            "   - Conclusions and Recommendation </policy>\n"
            )

In [186]:
user_input = ("<issue>Unable to re-register storage provider, vvol datastore is inaccessable</issue>"
              "<issue_confirmation> Confirmed over remote session </issue_confirmation>"
              "<cause_identification> vvol datastores are inaccessible, storage provider not connected, "
              " remove the storage providers, and try to re-register the storage provider "
              " but failed with unknown error. Tried using PURE plugin as well as manual process, "
              " didn't worked. Reset vasa cert at storage end, still no success, "
              " facing same problem with multiple storages </cause_identification>"
              "<cause_justification> We checked vvold.log "
              " We regenerated SMS certificate : service-control --stop vmware-sps "
              " /usr/lib/vmware-vmafd/bin/vecs-cli entry delete --store sms --alias sms_self_signed "
              " service-control --start vmware-sps "
              " We unregistered the storage provider using unreg_vasa script. "
              " python unreg_vasa.py -s <VC_IP_ADDRESS> "
              " vmon-cli -r sps "
              " Removed the Pure Storage plugin from mob page: "
              " Followed following article : "
              " https://knowledge.broadcom.com/external/article/368313/registering-vasa-provider-fails-with-err.html "
              " Issue got fixed: "
              " Then we had HA issues : Unable to find fdm agent. "
              " Checked vmdird and it was normal. "
              " Obtain current vCenter LDU ID with command: "
              " # /usr/lib/vmware-vmafd/bin/vmafd-cli get-ldu --server-name localhost "
              " Derived service ID from /var/log/firstboot/vpxd_firstboot_stdout file "
              " Re-assigned the license. "
              " Disabled and re-enabled HA. "
              " Issue resolved and customer confirmed case archival. </cause_justification>"
              "<solution_recommendation> "
              "  Re-assigned the license. "
              " Disabled and re-enabled HA. Issue resolved and customer confirmed case archival.</solution_recommendation>"
              "<solution_justification> "
              " we've also added the PURE storage plugin connections back to vCenter and so far everything looks stable. We'll delete all the snapshots of the vCenter after 24-hours of monitoring.   "
              " With this, you may soft archive the case for now.   </solution_justification>"
             )

In [174]:
print(structured_prompt)

<instructions>You are a support account manager. Your job is to create a root cause   analysis following critical system down event and share it with the customers.</instructions>
<policy>**The root cause analysis contains**

1. **Executive Summary**
   - Include high-level and concise issue description, business impact and resolution 
2. **Technical Findings and Root cause**
   - Symptoms 
   - Analysis 
   - Resolution/Workaround 
   - Issue Timeline 
   - Conclusions and Recommendation </policy>



In [176]:
print(user_input)

<issue>After MCP vCenter Upgrade to 8 VRA is failing to connect to vCenters(UAT & PROD) in cloud Accounts</issue><issue_confirmation> Confirmed over remote session </issue_confirmation><cause_identification> Failed to validate Credentials. Error failed to connect to vcenter error:  unable to tunnel through proxy. Proxy returns Http/1.1 500 Internal server Error. </cause_identification><cause_justification> On the call we observed we had issue with the cloud accounts :  Failed to validate Credentials. Error failed to connect to vcenter error:  unable to tunnel through proxy. Proxy returns "Http/1.1 500 Internal server Error"  We had issue syncing all cloud accounts since vCenter upgrade on Management node to 8.0.x.  We observed the below proxy set in the Aria Automation.  The upstream_proxy_host was set to 10.65.8.164 which resolved to a Jumpbox host. </cause_justification><solution_recommendation>   We deleted the proxy and set it to a default proxy.  After resetting the Aria Automatio

In [188]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": structured_prompt + user_input
                                          }]
                                         )
# Extract the response content
response_text = response.choices[0].message.content

# Remove Markdown formatting
plain_text = re.sub(r'[#*`_>-]', '', response_text)

# Print the cleaned-up text
print(plain_text)



 Root Cause Analysis

 1. Executive Summary

On [Date of Incident], a critical system outage occurred affecting the accessibility of VVol datastores within the customer's VMware environment. This issue prevented the reregistration of the storage provider, leading to significant business impact by disrupting data storage and virtual machine operations. Through a series of technical interventions, including certificate regeneration and license reassignment, the issue was successfully resolved, restoring full system functionality. The resolution was confirmed by the customer, and measures have been put in place to prevent future occurrences.

 2. Technical Findings and Root Cause

 Symptoms

 Inability to reregister the storage provider.
 VVol datastores became inaccessible.
 Encountered errors during the reregistration process using both PURE plugin and manual methods.

 Analysis

Upon investigation, the following findings were identified:

1. Storage Provider Connectivity Issues:
    